In [ ]:
def getting_comments_in_videos(youtube, video_ids):
    all_comments = {}

    for video_id in video_ids:
        try:
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id
            )
            response = request.execute()
            comments_in_video = []

            for comment in response.get('items', []):
                snippet = comment.get('snippet', {})
                topLevelComment = snippet.get('topLevelComment', {})
                topLevelSnippet = topLevelComment.get('snippet', {})
                text = topLevelSnippet.get('textOriginal', None)

                if text is not None:
                    comments_in_video.append(text)

            comments_in_video_information = {
                'video_id': video_id,
                'comments': comments_in_video
            }

            all_comments.append(comments_in_video_information)

        except:
          print(f"could not find the comments for this video alone{comments_in_video_information}")


    #return pd.DataFrame(all_comments)
    return(all_comments)

In [ ]:
def give_whole_video_details(youtube, channel_ids, playlist_ids, video_ids):
    channel_details = getting_channel_stats(youtube, channel_ids)
    playlist_id = getting_video_ids(youtube, playlist_ids)
    video_info = get_video_details(youtube, video_ids)
    comments = getting_comments_in_videos(youtube, video_ids)

    total_details = [{f"Video_Id_{count}": {**info, "comments": comment}} for count, (info, comment) in enumerate(zip(video_info.values(), comments.values()), start=1)]

    output = {**channel_details, **total_details}

    return output


In [ ]:
#This function is used to give all the details of the below functions
#This function gives statistics and details of the channels using the api and channelids
#This function gives the video IDs from the specified playlists
#This function gives the details and statistics of the videos
# This function gives  the comments for each video

In [ ]:
# mongodb connection

client = pymongo.MongoClient("your connection string")
upload=give_whole_video_details(API_KEY,channel_ids)
youtube_db= youtube_obj.("database_name")  #database in the mongodb
collection = youtube_db.("collection_name") # collection

try:
     existing_document=collection.find_one({"Channel_Name.Channel_Ids": channel_ids})
   if existing_document is None:
       collection.insert_one(upload)
             print("Data uploaded successfully")
    else:
        print("Channel ID already exists")

   except pymongo.errors.PyMongoError as e:
    print(f"Error occurred while uploading the data: ")
  finally:
           youtube_obj.close()